<a href="https://colab.research.google.com/github/AndresPautrat/MachineLearning_UPC/blob/main/TP_SKLearn_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install plotly==4.6.0

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
fileUp = files.upload()

Saving confirmed_global.csv to confirmed_global.csv


In [ ]:
data_set = pd.read_csv(io.BytesIO(fileUp['confirmed_global.csv']))

#Limpieza de datos

In [ ]:
dt=data_set.copy()

In [ ]:
dt = dt.sort_values(by=['4/24/20'], ascending=False)
#dt = dt.drop(dt.index[0:5], axis = 0)
#dt = dt.drop(dt.index[-5:], axis = 0)

In [ ]:
dt

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20
225,NaN,US,37.0902,-95.7129,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,...,4632,6421,7783,13747,19273,25600,33276,43843,53736,65778,83836,101657,121465,140909,161831,188172,213242,243622,275367,308650,336802,366317,397121,428654,462780,496535,526396,555313,580619,607670,636350,667592,699706,732197,758809,784326,811865,840351,869170,905358
201,NaN,Spain,40.0000,-4.0000,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,13,...,9942,11748,13910,17963,20410,25374,28768,35136,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065,119199,126168,131646,136675,141942,148220,153222,158273,163027,166831,170099,172541,177644,184948,190839,191726,198674,200210,204178,208389,213024,219764
137,NaN,Italy,43.0000,12.0000,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,...,27980,31506,35713,41035,47021,53578,59138,63927,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,192994
116,NaN,France,46.2276,2.2137,0,0,2,3,3,3,4,5,5,5,6,6,6,6,6,6,6,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,14,18,...,6633,7652,9043,10871,12612,14282,16018,19856,22304,25233,29155,32964,37575,40174,44550,52128,56989,59105,64338,68605,70478,74390,78167,82048,86334,90676,93790,120633,124298,129257,132473,144944,146923,146906,151808,154188,156921,154715,157026,158636
120,NaN,Germany,51.0000,9.0000,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,...,7272,9257,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Tibet,China,31.6927,88.0924,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
238,Recovered,Canada,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231,Diamond Princess,Canada,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0
37,Grand Princess,Canada,37.6489,-122.6655,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,8,9,9,10,10,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,-1,-1,-1


In [ ]:
dt=dt.drop(["Province/State","Country/Region","Lat","Long"],axis=1)

In [ ]:
negatives=[]
for i in range(dt.shape[0]):
  for j in range(dt.shape[1]):
    if(dt.iloc[i,j]<0):
      print(i)
      negatives+=[i]
      break;

261
262
263


In [ ]:
dt = dt.drop( dt.index[negatives])

In [ ]:
dt

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20
225,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,58,60,68,74,...,4632,6421,7783,13747,19273,25600,33276,43843,53736,65778,83836,101657,121465,140909,161831,188172,213242,243622,275367,308650,336802,366317,397121,428654,462780,496535,526396,555313,580619,607670,636350,667592,699706,732197,758809,784326,811865,840351,869170,905358
201,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,13,15,32,45,84,...,9942,11748,13910,17963,20410,25374,28768,35136,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065,119199,126168,131646,136675,141942,148220,153222,158273,163027,166831,170099,172541,177644,184948,190839,191726,198674,200210,204178,208389,213024,219764
137,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,655,888,1128,1694,...,27980,31506,35713,41035,47021,53578,59138,63927,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,192994
116,0,0,2,3,3,3,4,5,5,5,6,6,6,6,6,6,6,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,14,18,38,57,100,130,...,6633,7652,9043,10871,12612,14282,16018,19856,22304,25233,29155,32964,37575,40174,44550,52128,56989,59105,64338,68605,70478,74390,78167,82048,86334,90676,93790,120633,124298,129257,132473,144944,146923,146906,151808,154188,156921,154715,157026,158636
120,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,46,48,79,130,...,7272,9257,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
263,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
78,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# Reducción de dimensionalidad



In [ ]:
dt_temp=pd.DataFrame()
#for i in range(0,dt.shape[1],7):
for i in range(0,dt.shape[1],1):
  dt_temp["Semana " +str(i/7)]=dt.iloc[:,i]
dt=dt_temp.copy()

In [ ]:
dt_semanal=dt.copy()
dt.shape
for i in range(dt.shape[0]):
  for j in range(1,dt.shape[1]):
    dt_semanal.iloc[i,j]=dt.iloc[i,j]-dt.iloc[i,j-1]
dt=dt_semanal.copy()

In [ ]:
dt

,Semana 0.0,Semana 0.14285714285714285,Semana 0.2857142857142857,Semana 0.42857142857142855,Semana 0.5714285714285714,Semana 0.7142857142857143,Semana 0.8571428571428571,Semana 1.0,Semana 1.1428571428571428,Semana 1.2857142857142858,Semana 1.4285714285714286,Semana 1.5714285714285714,Semana 1.7142857142857142,Semana 1.8571428571428572,Semana 2.0,Semana 2.142857142857143,Semana 2.2857142857142856,Semana 2.4285714285714284,Semana 2.5714285714285716,Semana 2.7142857142857144,Semana 2.857142857142857,Semana 3.0,Semana 3.142857142857143,Semana 3.2857142857142856,Semana 3.4285714285714284,Semana 3.5714285714285716,Semana 3.7142857142857144,Semana 3.857142857142857,Semana 4.0,Semana 4.142857142857143,Semana 4.285714285714286,Semana 4.428571428571429,Semana 4.571428571428571,Semana 4.714285714285714,Semana 4.857142857142857,Semana 5.0,Semana 5.142857142857143,Semana 5.285714285714286,Semana 5.428571428571429,Semana 5.571428571428571,...,Semana 7.857142857142857,Semana 8.0,Semana 8.142857142857142,Semana 8.285714285714286,Semana 8.428571428571429,Semana 8.571428571428571,Semana 8.714285714285714,Semana 8.857142857142858,Semana 9.0,Semana 9.142857142857142,Semana 9.285714285714286,Semana 9.428571428571429,Semana 9.571428571428571,Semana 9.714285714285714,Semana 9.857142857142858,Semana 10.0,Semana 10.142857142857142,Semana 10.285714285714286,Semana 10.428571428571429,Semana 10.571428571428571,Semana 10.714285714285714,Semana 10.857142857142858,Semana 11.0,Semana 11.142857142857142,Semana 11.285714285714286,Semana 11.428571428571429,Semana 11.571428571428571,Semana 11.714285714285714,Semana 11.857142857142858,Semana 12.0,Semana 12.142857142857142,Semana 12.285714285714286,Semana 12.428571428571429,Semana 12.571428571428571,Semana 12.714285714285714,Semana 12.857142857142858,Semana 13.0,Semana 13.142857142857142,Semana 13.285714285714286,Target
225,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,58,60,68,74,...,6421,7783,13747,19273,25600,33276,43843,53736,65778,83836,101657,121465,140909,161831,188172,213242,243622,275367,308650,336802,366317,397121,428654,462780,496535,526396,555313,580619,607670,636350,667592,699706,732197,758809,784326,811865,840351,869170,905358,Aumenta
201,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,13,15,32,45,84,...,11748,13910,17963,20410,25374,28768,35136,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065,119199,126168,131646,136675,141942,148220,153222,158273,163027,166831,170099,172541,177644,184948,190839,191726,198674,200210,204178,208389,213024,219764,Aumenta
137,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,655,888,1128,1694,...,31506,35713,41035,47021,53578,59138,63927,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,192994,Aumenta
116,0,0,2,3,3,3,4,5,5,5,6,6,6,6,6,6,6,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,14,18,38,57,100,130,...,7652,9043,10871,12612,14282,16018,19856,22304,25233,29155,32964,37575,40174,44550,52128,56989,59105,64338,68605,70478,74390,78167,82048,86334,90676,93790,120633,124298,129257,132473,144944,146923,146906,151808,154188,156921,154715,157026,158636,Disminuye
120,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,46,48,79,130,...,9257,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999,Disminuye
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
temp=[]
for i in range(dt.shape[0]):
  if(dt.iloc[i,dt.shape[1]-1]-dt.iloc[i,dt.shape[1]-2]>0):
    temp+=["Aumenta"]
  elif(dt.iloc[i,dt.shape[1]-1]-dt.iloc[i,dt.shape[1]-2]<0):
    temp+=["Disminuye"]
  else:
    temp+=["Igual"]
dt_temp["Target"]=temp
dt=dt_temp

In [ ]:
dt=dt.drop(["Semana 13.0"],axis=1)

In [ ]:
dt.to_csv("dt_tratada.csv")

#Normalización de datos

In [ ]:
dt.iloc[:,:-1]=normalize(dt.iloc[:,:-1])

In [ ]:
dt

,Semana 0.0,Semana 1.0,Semana 2.0,Semana 3.0,Semana 4.0,Semana 5.0,Semana 6.0,Semana 7.0,Semana 8.0,Semana 9.0,Semana 10.0,Semana 11.0,Semana 12.0,Target
225,0.000001,0.000006,0.000014,0.000015,0.000016,0.000071,0.000186,0.001603,0.009740,0.082316,0.266856,0.536427,0.796342,Disminuye
201,0.000000,0.000000,0.000004,0.000008,0.000008,0.000050,0.000858,0.008796,0.053732,0.191268,0.402190,0.572549,0.686209,Aumenta
137,0.000000,0.000000,0.000008,0.000012,0.000012,0.001764,0.012032,0.048540,0.139102,0.289734,0.430686,0.543049,0.643279,Disminuye
116,0.000000,0.000030,0.000036,0.000065,0.000071,0.000107,0.001696,0.013571,0.053800,0.150121,0.339049,0.488135,0.788132,Disminuye
120,0.000000,0.000020,0.000061,0.000081,0.000081,0.000137,0.001333,0.009711,0.062737,0.189952,0.396322,0.576609,0.685812,Disminuye
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.426401,0.639602,0.639602,Igual
259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.707107,Igual
263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Disminuye
78,0.000000,0.000000,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,0.301511,Igual


In [ ]:
#dt.to_csv('dt_tratada.csv')

In [ ]:
x2 = dt.iloc[:,:-1]

In [ ]:
pca = PCA(2)
principal_Components = pca.fit_transform(x2)
data_PCA = pd.DataFrame(data = principal_Components, columns=['PCA1','PCA2'])
data_PCA.shape

(261, 2)

In [ ]:
data_PCA = pd.concat([data_PCA,dt['Target']], axis=1)
data_PCA.shape

(264, 3)

In [ ]:
data_PCA.sort_values(by='Target')

,PCA1,PCA2,Target
0,-0.177984,0.035936,Aumenta
124,-0.069604,-0.082621,Aumenta
126,-0.000798,-0.199370,Aumenta
127,0.014451,-0.242858,Aumenta
131,-0.049042,-0.194848,Aumenta
...,...,...,...
113,0.673964,0.046731,Igual
234,-0.278616,0.333246,Igual
37,-0.056614,-0.076430,NaN
41,-0.146539,-0.016564,NaN


#Visualización de datos

In [ ]:
x = dt.iloc[:,:-1].values
y = dt.iloc[:,-1].values

In [ ]:
dt_semanal=dt_semanal.sort_values(by=['Semana 13.0'], ascending=False)

In [ ]:
dt_temp_trans=dt_semanal.transpose() 

In [ ]:
fig = go.Figure()
count=0
for i in dt_temp_trans:
  fig.add_trace(go.Scatter(x=dt_temp_trans.index.values, y=dt_temp_trans[i],
                    mode='lines+markers',
                    name=data_set["Country/Region"][i]+"-"+str(i)))
  count+=1

fig.show()

In [ ]:
temp_graphs=[]
for i in dt_semanal.index:
  temp_graphs+=[go.Bar(name=data_set["Country/Region"][i]+"-"+str(i), x=dt_semanal.columns, y=dt_semanal.loc[i,:])]
fig = go.Figure(data=temp_graphs)
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
total = sum(dt["Target"].value_counts())
percentage = [(dt["Target"].value_counts()[i]/total)*100 for i in range(3)]
langs = dt["Target"].value_counts().keys().tolist()
fig = px.pie(dt, values=percentage, names=langs)
fig.show()

In [ ]:
temp_dt=data_set.copy()
temp_dt = temp_dt.sort_values(by =['4/24/20'],ascending=False)
temp_dt=temp_dt.drop( temp_dt.index[negatives])
temp_dt=temp_dt.drop(negatives)
fig = px.scatter_geo(temp_dt, lat="Lat", lon="Long",
                     size="4/24/20", hover_name="Country/Region"
                     )
fig.show()

#Tecnicas de muestreo

#Redes bayesianas GaussianNB

In [ ]:
x = data_PCA.iloc[:,:-1]
y = data_PCA.iloc[:,-1]
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.3)

In [ ]:
clf = GaussianNB()
clf.fit(X_train,Y_train)

TypeError: ignored

In [ ]:
clf.predict(X_test)

In [ ]:
predicciones = clf.predict(X_test)
print("Predicciones",accuracy_score(Y_test, predicciones))
print(confusion_matrix(Y_test, predicciones))
print(classification_report(Y_test, predicciones))

In [ ]:
def crossValidation(model,x,y):  
  cross = []
  for i in range(10,40,4):
    cross.append(cross_val_score(model, x, y ,cv=i).mean()) 
  return cross

In [ ]:
crossValidation(clf,x,y)

In [ ]:
def crossValidationStrati(model):
  x = data_PCA.iloc[:,:-1].values
  y = data_PCA.iloc[:,-1].values
  acurracy = []
  acurracy1 = []
  for i in range(10,30,3):
    skf = StratifiedKFold(n_splits=i, random_state=None, shuffle=True)
    for train_index, test_index in skf.split(x, y):
      X_train, X_test = x[train_index], x[test_index]
      y_train, y_test = y[train_index], y[test_index]
      
      model.fit(X_train, y_train)
      predicc = model.predict(X_test)
      acurracy1.append(accuracy_score(y_test, predicc))
    acurracy.append(acurracy1)
    acurracy1 = []
  suma = 0
  for item in acurracy:
    for a in item:
      suma = suma + a
    print(suma/len(item))
    suma = 0 

In [ ]:
crossValidationStrati(clf)

#Arboles de desición


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.3)

TreeClass = DecisionTreeClassifier(random_state=0)
TreeClass.fit(X_train,Y_train)

In [ ]:
TreeClass.predict(X_test)

In [ ]:
prediccionesTree = TreeClass.predict(X_test)
print("prediccion",accuracy_score(Y_test, prediccionesTree))
print(confusion_matrix(Y_test, prediccionesTree))
print(classification_report(Y_test, prediccionesTree))

In [ ]:
crossValidation(TreeClass,x,y)

In [ ]:
crossValidationStrati(TreeClass)

#Redes neuronales

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.3)
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(X_train, Y_train)

In [ ]:
prediccionesMLP = mlp.predict(X_test)
print(accuracy_score(Y_test, prediccionesMLP))

In [ ]:
print("prediccion",accuracy_score(Y_test, prediccionesMLP))
print(confusion_matrix(Y_test, prediccionesMLP))
print(classification_report(Y_test, prediccionesMLP))

In [ ]:
crossValidation(mlp,x,y)

In [ ]:
crossValidationStrati(mlp)

#Logistic Regression



In [ ]:
#y = data_PCA.iloc[:,-1].values
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.3)
linear = LogisticRegression(random_state=0)
linear.fit(X_train,Y_train)

In [ ]:
crossValidation(linear,x,y)

In [ ]:
crossValidationStrati(linear)